In [184]:
from platform import python_version

print(python_version())

3.10.2


In [185]:
import pandas as pd

import pubchempy as pcp
import numpy as np
def isNaN(string):
    return string != string
import os
import glob
import re
from pybatchclassyfire import *
import csv 
import time
import json
from pandas import json_normalize
import wget
import string
import urllib.parse
import openpyxl
import statistics
import sys
from itertools import chain

In [186]:
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import AllChem
from rdkit.Chem import rdFMCS
from rdkit.Chem import PandasTools

In [60]:
# write function for Spectral database results

In [187]:
input_dir = "/Users/mahnoorzulfiqar/OneDriveUNI/MAW-data/StandardSMarinoi_Data"
input_dir

'/Users/mahnoorzulfiqar/OneDriveUNI/MAW-data/StandardSMarinoi_Data'

In [188]:
entries = []
# entry is all files and folders in input_dir
for entry in os.listdir(input_dir):
     #if the entry is also a directory
    if os.path.isdir(os.path.join(input_dir, entry)):
        print(entry)
        entries.append(entry)

VN_211016_cyst_acid_PRM_neg
VN_211016_Sc_QC_PRM_neg
VN_211016_acetyl_carnitine
VN_211016_butanoyl_carnitine
VN_211016_methionine_sulfoxide
SL_Frag
VN_211016_pipecolate
VN_211016_Sc_st_PRM_pos
VN_211016_betaine
VN_211016_cyst_acid_PRM_pos
VN_211016_Sc_QC_PRM_pos
VN_211016_isovalerylcarnitine
VN_211016_propanoyl_carnitine
VN_211016_Sc_st_PRM_neg


In [189]:
entry = entries[2]
entry

'VN_211016_acetyl_carnitine'

In [190]:
# reach spectra_dereplication folder
sub_dir_spec = input_dir + "/" + entry + '/spectral_dereplication/'
# reach SIRIUS results
sub_dir_sir = input_dir + "/" + entry + '/insilico/SIRIUS/'

#list of all csv files in the spectral dereplication foler
spec_msp_csv = (glob.glob(input_dir + "/" + entry + '/spectral_dereplication' +'/*.csv'))
# Sirius csv result file
sir_msp_csv = input_dir + "/" + entry + "/insilico/MS1DATA.csv"

# if both exist; which should be the case, even in case of 0 results
if os.path.exists(sir_msp_csv) and os.path.exists(spec_msp_csv[0]):

    # read both csv files
    spec_msv = pd.read_csv(spec_msp_csv[0])
    sir_msv = pd.read_csv(sir_msp_csv)
    
    spec_msv = spec_msv[[
            'id_X',
            'premz',
            'rtmin',
            'rtmax',
            'rtmed',
            'rtmean',
            'col_eng',
            'pol',
            'int',
            'source_file',
            'mbank_results_csv',
            ]]
    sir_msv = sir_msv[['id_X',
            'premz',
            'rtmed',
            'rtmean',
            'int',
            'col_eng',
            'pol',
            'ms2Peaks',
            'ms1Peaks',
            'sirius_result_dir']]

    merged_df = sir_msv.merge(spec_msv, 
                      how='inner', 
                      left_on=['premz', 'rtmed','rtmean','int','col_eng','pol'], 
                      right_on=['premz','rtmed','rtmean','int','col_eng','pol'])




In [193]:
merged_df["sirius_result_dir"][0]

'./VN_211016_acetyl_carnitine/insilico/SIRIUS/results_for_1M204R149ID1_structure_1M204R149ID1.csv'

In [195]:
def chemMN_CandidateSelection(df):
    

    # define empty variable
    dbn= []
    for i, row in df.iterrows():
        # to compare each element with each opther element
        for j, row in df.iterrows():

            ms = [Chem.MolFromSmiles(df['SMILES'][i]), Chem.MolFromSmiles(df['SMILES'][j])]
            fps = [AllChem.GetMorganFingerprintAsBitVect(x,2, nBits=2048) for x in ms]
            tn = DataStructs.FingerprintSimilarity(fps[0],fps[1])
            dbn.append({
                'Name_i':df['ranks'][i],
                'Name_j':df['ranks'][j],
                'i': df['SMILES'][i],
                'j': df['SMILES'][j],
                'Source_i':df['Source'][i],
                'Source_j':df['Source'][j],
                'Tanimoto': tn
            })
        #except Exception as e:
            #pass
            #print(i)
            #print(j)
            #print(e)
    # save chemical similarities                    
    db_edgenode = pd.DataFrame(dbn)

    dfe = []
    heavy_atoms = ['C', 'N', 'P', 'O', 'S']
    for i, row in db_edgenode.iterrows():        
        if db_edgenode['Tanimoto'][i] >= 0.70:
            # list of mol used to calaculate the MCSS
            n = [Chem.MolFromSmiles(db_edgenode['i'][i]),Chem.MolFromSmiles(db_edgenode['j'][i])]
            res = rdFMCS.FindMCS(n)
            sm_res = Chem.MolToSmiles(Chem.MolFromSmarts(res.smartsString))
            # Check if the MCSS has one of the heavy atoms and whether they are
            # more than 3
            elem = [ele for ele in heavy_atoms if(ele in sm_res)]
            if elem and len(sm_res)>=3:
                MCSS_SMILES = Chem.MolToSmiles(Chem.MolFromSmarts(res.smartsString))

            dfe.append({
                'Start':db_edgenode['Name_i'][i],
                'End':db_edgenode['Name_j'][i],
                'Tanimoto':db_edgenode['Tanimoto'][i],
                'Start_SMILES':db_edgenode['i'][i],
                'End_SMILES':db_edgenode['j'][i],
                'Start_Source':db_edgenode['Source_i'][i],
                'End_Source':db_edgenode['Source_j'][i],
                'MCSS': MCSS_SMILES
            })
    df_edge = pd.DataFrame(dfe)
    df_edge['Start'] = df_edge['Start'].astype(str)
    df_edge['End'] = df_edge['End'].astype(str)
    df_edge['sorted_row'] = [sorted([a,b]) for a,b in zip(df_edge.Start,df_edge.End)]
    df_edge['sorted_row'] = df_edge['sorted_row'].astype(str)
    df_edge.drop_duplicates(subset=['sorted_row'], inplace=True)

    #nodes= []
    #for i, row in df.iterrows():
        #n = df['ranks'][i]
        #nodes.append({
            #'nodes':n
        #})

    node= pd.DataFrame(nodes)

    return(df_edge)

In [ ]:
chemMN_CandidateSelection(df)

In [194]:
# entry is all files and folders in input_dir
for entry in os.listdir(input_dir):
     #if the entry is also a directory
    if os.path.isdir(os.path.join(input_dir, entry)):


        # reach spectra_dereplication folder
        sub_dir_spec = input_dir + "/" + entry + '/spectral_dereplication/'
        # reach SIRIUS results
        sub_dir_sir = input_dir + "/" + entry + '/insilico/SIRIUS/'

        #list of all csv files in the spectral dereplication foler
        spec_msp_csv = (glob.glob(input_dir + "/" + entry + '/spectral_dereplication' +'/*.csv'))
        # Sirius csv result file
        sir_msp_csv = input_dir + "/" + entry + "/insilico/MS1DATA.csv"

        # if both exist; which should be the case, even in case of 0 results
        if os.path.exists(sir_msp_csv) and os.path.exists(spec_msp_csv[0]):

            # read both csv files
            spec_msv = pd.read_csv(spec_msp_csv[0])
            sir_msv = pd.read_csv(sir_msp_csv)



            spec_msv = spec_msv[[
                'id_X',
                'premz',
                'rtmin',
                'rtmax',
                'rtmed',
                'rtmean',
                'col_eng',
                'pol',
                'int',
                'source_file',
                'mbank_results_csv',
                ]]
            sir_msv = sir_msv[['id_X',
                'premz',
                'rtmed',
                'rtmean',
                'int',
                'col_eng',
                'pol',
                'ms2Peaks',
                'ms1Peaks',
                'sirius_result_dir']]

            merged_df = sir_msv.merge(spec_msv, 
                          how='inner', 
                          left_on=['premz', 'rtmed','rtmean','int','col_eng','pol'], 
                          right_on=['premz','rtmed','rtmean','int','col_eng','pol'])

            for mer, rows in merged_df.iterrows():

                sirius_csv = merged_df["sirius_result_dir"][mer].replace("./", input_dir+"/")
                mbank_csv = merged_df["mbank_results_csv"][mer].replace("./", input_dir+"/")
                gnps_csv = merged_df["mbank_results_csv"][mer].replace("./", input_dir+"/").replace('mbank', 'gnps').replace('MassBank', 'GNPS')
                hmdb_csv = merged_df["mbank_results_csv"][mer].replace("./", input_dir+"/").replace('mbank', 'hmdb').replace('MassBank', 'HMDB')

                
                if os.path.exists(sirius_csv) and os.path.exists(gnps_csv) and os.path.exists(mbank_csv) and os.path.exists(hmdb_csv):


                    sirius_df = pd.read_csv(sirius_csv)
                    sirius_df = sirius_df.drop_duplicates('smiles')
                    
                    mbank_df = pd.read_csv(mbank_csv)
                    mbank_df = mbank_df.drop_duplicates('MBSMILES')

                    gnps_df = pd.read_csv(gnps_csv)
                    gnps_df = gnps_df.drop_duplicates('GNPSSMILES')

                    hmdb_df = pd.read_csv(hmdb_csv)
                    hmdb_df = hmdb_df.drop_duplicates('HMDBSMILES')

                    #1 SGHM
                    if len(sirius_df) > 0 and len(gnps_df) > 0 and len(mbank_df) > 0 and len(hmdb_df) > 0:
                        mbank_df["rank_ids"] = ["M_" + str(s+1) for s in range(len(mbank_df))]

                        gnps_df["rank_ids"] = ["G_" + str(s+1) for s in range(len(gnps_df))]
                        
                        hmdb_df["rank_ids"] = ["H_" + str(s+1) for s in range(len(hmdb_df))]

                        sirius_df["rank_ids"] = ["S_" + str(s) for s in sirius_df["rank"]]
                        sirius_df["Source"] = "SIRIUS"


                        source_l1 = [*(list(sirius_df["Source"])), 
                           *(list(gnps_df["Source"]))
                           ,*(list(mbank_df["Source"])),
                                    *(list(hmdb_df["Source"]))]

                        rank_l2 = [*(list(sirius_df["rank_ids"])), 
                           *(list(gnps_df["rank_ids"]))
                           ,*(list(mbank_df["rank_ids"])), 
                                  *(list(hmdb_df["rank_ids"]))]

                        smiles_l3 = [*(list(sirius_df["smiles"])), 
                           *(list(gnps_df["GNPSSMILES"]))
                           ,*(list(mbank_df["MBSMILES"])),
                                    *(list(hmdb_df["HMDBSMILES"]))]

                        sm = pd.DataFrame(list(zip(source_l1, rank_l2, smiles_l3)), columns = ["Source", "ranks", "SMILES"])

                        df_edge = chemMN_CandidateSelection(sm)
                        
                        df_edge.to_csv(input_dir + "/" + entry + "/ChemMNedges.tsv", sep='\t')

                    
                    #2 SGM
                    elif  len(sirius_df) > 0 and len(gnps_df) > 0 and len(mbank_df) > 0 and len(hmdb_df) == 0:
                        
                        mbank_df["rank_ids"] = ["M_" + str(s+1) for s in range(len(mbank_df))]

                        gnps_df["rank_ids"] = ["G_" + str(s+1) for s in range(len(gnps_df))]

                        sirius_df["rank_ids"] = ["S_" + str(s) for s in sirius_df["rank"]]
                        sirius_df["Source"] = "SIRIUS"


                        source_l1 = [*(list(sirius_df["Source"])), 
                           *(list(gnps_df["Source"]))
                           ,*(list(mbank_df["Source"]))]

                        rank_l2 = [*(list(sirius_df["rank_ids"])), 
                           *(list(gnps_df["rank_ids"]))
                           ,*(list(mbank_df["rank_ids"]))]

                        smiles_l3 = [*(list(sirius_df["smiles"])), 
                           *(list(gnps_df["GNPSSMILES"]))
                           ,*(list(mbank_df["MBSMILES"]))]

                        sm = pd.DataFrame(list(zip(source_l1, rank_l2, smiles_l3)), columns = ["Source", "ranks", "SMILES"])


                        df_edge = chemMN_CandidateSelection(sm)

                        df_edge.to_csv(input_dir + "/" + entry + "/ChemMNedges.tsv", sep='\t')
                        
                    #3 SHM
                    elif  len(sirius_df) > 0 and len(gnps_df) == 0 and len(mbank_df) > 0 and len(hmdb_df) > 0:

                        mbank_df["rank_ids"] = ["M_" + str(s+1) for s in range(len(mbank_df))]

                        #gnps_df["rank_ids"] = ["G_" + str(s+1) for s in range(len(gnps_df))]
                        
                        hmdb_df["rank_ids"] = ["H_" + str(s+1) for s in range(len(hmdb_df))]

                        sirius_df["rank_ids"] = ["S_" + str(s) for s in sirius_df["rank"]]
                        sirius_df["Source"] = "SIRIUS"


                        source_l1 = [*(list(sirius_df["Source"]))
                           ,*(list(mbank_df["Source"])),
                                    *(list(hmdb_df["Source"]))]

                        rank_l2 = [*(list(sirius_df["rank_ids"]))
                           ,*(list(mbank_df["rank_ids"])), 
                                  *(list(hmdb_df["rank_ids"]))]

                        smiles_l3 = [*(list(sirius_df["smiles"]))
                           ,*(list(mbank_df["MBSMILES"])),
                                    *(list(hmdb_df["HMDBSMILES"]))]

                        sm = pd.DataFrame(list(zip(source_l1, rank_l2, smiles_l3)), columns = ["Source", "ranks", "SMILES"])


                        df_edge = chemMN_CandidateSelection(sm)

                        df_edge.to_csv(input_dir + "/" + entry + "/ChemMNedges.tsv", sep='\t')

                        
                    #4 SGH
                    elif  len(sirius_df) > 0 and len(gnps_df) > 0 and len(mbank_df) == 0 and len(hmdb_df) > 0:
                        #mbank_df["rank_ids"] = ["M_" + str(s+1) for s in range(len(mbank_df))]

                        gnps_df["rank_ids"] = ["G_" + str(s+1) for s in range(len(gnps_df))]
                        
                        hmdb_df["rank_ids"] = ["H_" + str(s+1) for s in range(len(hmdb_df))]

                        sirius_df["rank_ids"] = ["S_" + str(s) for s in sirius_df["rank"]]
                        sirius_df["Source"] = "SIRIUS"


                        source_l1 = [*(list(sirius_df["Source"]))
                           ,*(list(gnps_df["Source"])),
                                    *(list(hmdb_df["Source"]))]

                        rank_l2 = [*(list(sirius_df["rank_ids"]))
                           ,*(list(gnps_df["rank_ids"])), 
                                  *(list(hmdb_df["rank_ids"]))]

                        smiles_l3 = [*(list(sirius_df["smiles"]))
                           ,*(list(gnps_df["GNPSSMILES"])),
                                    *(list(hmdb_df["HMDBSMILES"]))]

                        sm = pd.DataFrame(list(zip(source_l1, rank_l2, smiles_l3)), columns = ["Source", "ranks", "SMILES"])

                        df_edge = chemMN_CandidateSelection(sm)

                        df_edge.to_csv(input_dir + "/" + entry + "/ChemMNedges.tsv", sep='\t')
                    #5 GHM
                    elif  len(sirius_df) == 0 and len(gnps_df) > 0 and len(mbank_df) > 0 and len(hmdb_df) > 0:
                        mbank_df["rank_ids"] = ["M_" + str(s+1) for s in range(len(mbank_df))]

                        gnps_df["rank_ids"] = ["G_" + str(s+1) for s in range(len(gnps_df))]
                        
                        hmdb_df["rank_ids"] = ["H_" + str(s+1) for s in range(len(hmdb_df))]

                        #sirius_df["rank_ids"] = ["S_" + str(s) for s in sirius_df["rank"]]
                        #sirius_df["Source"] = "SIRIUS"


                        source_l1 = [*(list(gnps_df["Source"]))
                           ,*(list(mbank_df["Source"])),
                                    *(list(hmdb_df["Source"]))]

                        rank_l2 = [*(list(gnps_df["rank_ids"]))
                           ,*(list(mbank_df["rank_ids"])), 
                                  *(list(hmdb_df["rank_ids"]))]

                        smiles_l3 = [*(list(gnps_df["GNPSSMILES"]))
                           ,*(list(mbank_df["MBSMILES"])),
                                    *(list(hmdb_df["HMDBSMILES"]))]

                        sm = pd.DataFrame(list(zip(source_l1, rank_l2, smiles_l3)), columns = ["Source", "ranks", "SMILES"])

                        df_edge = chemMN_CandidateSelection(sm)

                        df_edge.to_csv(input_dir + "/" + entry + "/ChemMNedges.tsv", sep='\t')
                    #6 SG
                    elif  len(sirius_df) > 0 and len(gnps_df) > 0 and len(mbank_df) == 0 and len(hmdb_df) == 0:
                        #mbank_df["rank_ids"] = ["M_" + str(s+1) for s in range(len(mbank_df))]

                        gnps_df["rank_ids"] = ["G_" + str(s+1) for s in range(len(gnps_df))]

                        sirius_df["rank_ids"] = ["S_" + str(s) for s in sirius_df["rank"]]
                        sirius_df["Source"] = "SIRIUS"


                        source_l1 = [*(list(sirius_df["Source"])), 
                           *(list(gnps_df["Source"]))]
                           #,*(list(mbank_df["Source"]))]

                        rank_l2 = [*(list(sirius_df["rank_ids"])), 
                           *(list(gnps_df["rank_ids"]))]
                           #,*(list(mbank_df["rank_ids"]))]

                        smiles_l3 = [*(list(sirius_df["smiles"])), 
                           *(list(gnps_df["GNPSSMILES"]))]
                           #,*(list(mbank_df["MBSMILES"]))]

                        sm = pd.DataFrame(list(zip(source_l1, rank_l2, smiles_l3)), columns = ["Source", "ranks", "SMILES"])


                        df_edge = chemMN_CandidateSelection(sm)

                        df_edge.to_csv(input_dir + "/" + entry + "/ChemMNedges.tsv", sep='\t')
                        
                    #7 SH
                    elif  len(sirius_df) > 0 and len(gnps_df) == 0 and len(mbank_df) == 0 and len(hmdb_df) > 0:
                        #mbank_df["rank_ids"] = ["M_" + str(s+1) for s in range(len(mbank_df))]

                        #gnps_df["rank_ids"] = ["G_" + str(s+1) for s in range(len(gnps_df))]
                        
                        hmdb_df["rank_ids"] = ["H_" + str(s+1) for s in range(len(hmdb_df))]

                        sirius_df["rank_ids"] = ["S_" + str(s) for s in sirius_df["rank"]]
                        sirius_df["Source"] = "SIRIUS"


                        source_l1 = [*(list(sirius_df["Source"])),
                                    *(list(hmdb_df["Source"]))]

                        rank_l2 = [*(list(sirius_df["rank_ids"])), 
                                  *(list(hmdb_df["rank_ids"]))]

                        smiles_l3 = [*(list(sirius_df["smiles"])),
                                    *(list(hmdb_df["HMDBSMILES"]))]

                        sm = pd.DataFrame(list(zip(source_l1, rank_l2, smiles_l3)), columns = ["Source", "ranks", "SMILES"])
                        df_edge = chemMN_CandidateSelection(sm)

                        df_edge.to_csv(input_dir + "/" + entry + "/ChemMNedges.tsv", sep='\t')
                    #8 SM
                    elif  len(sirius_df) > 0 and len(gnps_df) == 0 and len(mbank_df) > 0 and len(hmdb_df) == 0:
                        mbank_df["rank_ids"] = ["M_" + str(s+1) for s in range(len(mbank_df))]

                        #gnps_df["rank_ids"] = ["G_" + str(s+1) for s in range(len(gnps_df))]
                        
                        #hmdb_df["rank_ids"] = ["H_" + str(s+1) for s in range(len(hmdb_df))]

                        sirius_df["rank_ids"] = ["S_" + str(s) for s in sirius_df["rank"]]
                        sirius_df["Source"] = "SIRIUS"


                        source_l1 = [*(list(sirius_df["Source"]))
                           ,*(list(mbank_df["Source"]))]

                        rank_l2 = [*(list(sirius_df["rank_ids"]))
                           ,*(list(mbank_df["rank_ids"]))]

                        smiles_l3 = [*(list(sirius_df["smiles"]))
                           ,*(list(mbank_df["MBSMILES"]))]

                        sm = pd.DataFrame(list(zip(source_l1, rank_l2, smiles_l3)), columns = ["Source", "ranks", "SMILES"])


                        df_edge = chemMN_CandidateSelection(sm)

                        df_edge.to_csv(input_dir + "/" + entry + "/ChemMNedges.tsv", sep='\t')
                    #9 GM
                    elif  len(sirius_df) == 0 and len(gnps_df) > 0 and len(mbank_df) > 0 and len(hmdb_df) == 0:
                        mbank_df["rank_ids"] = ["M_" + str(s+1) for s in range(len(mbank_df))]

                        gnps_df["rank_ids"] = ["G_" + str(s+1) for s in range(len(gnps_df))]
                        
                        #hmdb_df["rank_ids"] = ["H_" + str(s+1) for s in range(len(hmdb_df))]

                        #sirius_df["rank_ids"] = ["S_" + str(s) for s in sirius_df["rank"]]
                        #sirius_df["Source"] = "SIRIUS"


                        source_l1 = [*(list(mbank_df["Source"])),
                                    *(list(gnps_df["Source"]))]

                        rank_l2 = [*(list(mbank_df["rank_ids"])), 
                                  *(list(gnps_df["rank_ids"]))]

                        smiles_l3 = [*(list(mbank_df["MBSMILES"])),
                                    *(list(gnps_df["GNPSSMILES"]))]

                        sm = pd.DataFrame(list(zip(source_l1, rank_l2, smiles_l3)), columns = ["Source", "ranks", "SMILES"])


                        
                        
                        df_edge = chemMN_CandidateSelection(sm)

                        df_edge.to_csv(input_dir + "/" + entry + "/ChemMNedges.tsv", sep='\t')
                    #10 GH
                    elif  len(sirius_df) == 0 and len(gnps_df) > 0 and len(mbank_df) == 0 and len(hmdb_df) > 0:
                        #mbank_df["rank_ids"] = ["M_" + str(s+1) for s in range(len(mbank_df))]

                        gnps_df["rank_ids"] = ["G_" + str(s+1) for s in range(len(gnps_df))]
                        
                        hmdb_df["rank_ids"] = ["H_" + str(s+1) for s in range(len(hmdb_df))]

                        #sirius_df["rank_ids"] = ["S_" + str(s) for s in sirius_df["rank"]]
                        #sirius_df["Source"] = "SIRIUS"


                        source_l1 = [*(list(gnps_df["Source"])),
                                    *(list(hmdb_df["Source"]))]

                        rank_l2 = [*(list(gnps_df["rank_ids"])),
                                  *(list(hmdb_df["rank_ids"]))]

                        smiles_l3 = [*(list(gnps_df["GNPSSMILES"])),
                                    *(list(hmdb_df["HMDBSMILES"]))]

                        sm = pd.DataFrame(list(zip(source_l1, rank_l2, smiles_l3)), columns = ["Source", "ranks", "SMILES"])
                        df_edge = chemMN_CandidateSelection(sm)


                        
                        df_edge.to_csv(input_dir + "/" + entry + "/ChemMNedges.tsv", sep='\t')
                    #11 HM
                    elif  len(sirius_df) == 0 and len(gnps_df) == 0 and len(mbank_df) > 0 and len(hmdb_df) > 0:
                        mbank_df["rank_ids"] = ["M_" + str(s+1) for s in range(len(mbank_df))]

                        #gnps_df["rank_ids"] = ["G_" + str(s+1) for s in range(len(gnps_df))]
                        
                        hmdb_df["rank_ids"] = ["H_" + str(s+1) for s in range(len(hmdb_df))]

                        #sirius_df["rank_ids"] = ["S_" + str(s) for s in sirius_df["rank"]]
                        #sirius_df["Source"] = "SIRIUS"


                        source_l1 = [*(list(mbank_df["Source"])),
                                    *(list(hmdb_df["Source"]))]

                        rank_l2 = [*(list(mbank_df["rank_ids"])), 
                                  *(list(hmdb_df["rank_ids"]))]

                        smiles_l3 = [*(list(mbank_df["MBSMILES"])),
                                    *(list(hmdb_df["HMDBSMILES"]))]

                        sm = pd.DataFrame(list(zip(source_l1, rank_l2, smiles_l3)), columns = ["Source", "ranks", "SMILES"])


                        df_edge = chemMN_CandidateSelection(sm)

                        df_edge.to_csv(input_dir + "/" + entry + "/ChemMNedges.tsv", sep='\t')
                        
                    # S
                    #elif  len(sirius_df) > 0 and len(gnps_df) == 0 and len(mbank_df) == 0 and len(hmdb_df) == 0:
                        
                    # G
                    #elif  len(sirius_df) == 0 and len(gnps_df) > 0 and len(mbank_df) == 0 and len(hmdb_df) == 0:
                        
                    # M
                    #elif  len(sirius_df) == 0 and len(gnps_df) == 0 and len(mbank_df) > 0 and len(hmdb_df) == 0:
                        
                    # H
                    #elif  len(sirius_df) == 0 and len(gnps_df) == 0 and len(mbank_df) == 0 and len(hmdb_df) > 0:

IndentationError: expected an indented block after 'elif' statement on line 278 (2652776907.py, line 280)

In [188]:
for smiles, rows in sm.iterrows():
    
    for sirius_i, row in sirius_df.iterrows():
        
        ms = [Chem.MolFromSmiles(sm["SMILES"][smiles]), Chem.MolFromSmiles(sirius_df["smiles"][sirius_i])]
        fps = [AllChem.GetMorganFingerprintAsBitVect(x,2, nBits=2048) for x in ms]
        tn = DataStructs.FingerprintSimilarity(fps[0],fps[1])
        if tn >= 0.99:
            if isNaN(sm["SIRIUS"][smiles]):
            
                sm.loc[smiles, "SIRIUS"] = str(sirius_i)

            else:
                sm.loc[smiles, "SIRIUS"] = str(sm["SIRIUS"][smiles]) + ", "+ str(sirius_i)
                
                
    for gnps_i, row in gnps_df.iterrows():
        
        ms = [Chem.MolFromSmiles(sm["SMILES"][smiles]), Chem.MolFromSmiles(gnps_df["GNPSSMILES"][gnps_i])]
        fps = [AllChem.GetMorganFingerprintAsBitVect(x,2, nBits=2048) for x in ms]
        tn = DataStructs.FingerprintSimilarity(fps[0],fps[1])
        
        if tn >= 0.99:
            if isNaN(sm["GNPS"][smiles]):

                sm.loc[smiles, "GNPS"] = str(gnps_i)

            else:
                sm.loc[smiles, "GNPS"] = str(sm["GNPS"][smiles]) + ", "+ str(gnps_i)
            
    for mbank_i, row in mbank_df.iterrows():
        ms = [Chem.MolFromSmiles(sm["SMILES"][smiles]), Chem.MolFromSmiles(mbank_df["MBSMILES"][mbank_i])]
        fps = [AllChem.GetMorganFingerprintAsBitVect(x,2, nBits=2048) for x in ms]
        tn = DataStructs.FingerprintSimilarity(fps[0],fps[1])
        if tn >= 0.99:
            if isNaN(sm["MassBank"][smiles]):

                sm.loc[smiles, "MassBank"] = str(mbank_i)

            else:
                sm.loc[smiles, "MassBank"] = str(sm["MassBank"][smiles]) + ", "+ str(mbank_i)

In [189]:
sm # discuss with Maria tomorrow

,SMILES,SIRIUS,GNPS,MassBank
0,CC(=O)OC(CC(=O)O)C[N+](C)(C)C,0,3,NaN
1,CCC(=O)OC(CC(=O)O)CN(C)C,1,NaN,NaN
2,CC(=O)OC(CC(=O)OC)CN(C)C,2,NaN,NaN
3,CC(=O)OC(C[N+](C)(C)C)OC(=O)C,3,NaN,NaN
4,CC(=O)OC(=O)CC(C[N+](C)(C)C)O,4,NaN,NaN
5,CC(=O)O[C@H](CC(=O)[O-])C[N+](C)(C)C,NaN,"0, 1, 4",0
6,CC(=O)OC(CC([O-])=O)C[N+](C)(C)C,NaN,"0, 1, 4",0
7,CC(=O)O[C@H](CC(O)=O)C[N+](C)(C)C,0,3,NaN
8,CC(=O)OC(CC(=O)[O-])C[N+](C)(C)C,NaN,"0, 1, 4",0
9,CC(=O)O[C@@H](CC(=O)[O-])C[N+](C)(C)C,NaN,"0, 1, 4",0
